In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob

import pandas as pd

In [3]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/binance','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())

        #d['adgt_%'] = d['adgl_%']+d['adgs_%']
        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_102576/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_102576/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_102576/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_102576/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs
0,\1000BTTCUSDT\plots\2022-03-15T184749\backtest...,5.5,1000BTTCUSDT,True,3.273,0.05900,100.0,0.9755,True,5.101,0.1270,100.000,0.98140
1,\1000XECUSDT\plots\2022-03-15T184506\backtest_...,5.5,1000XECUSDT,True,13.140,0.06520,100.0,0.9340,True,29.910,0.1540,100.000,0.95260
2,\AAVEUSDT\plots\2022-03-14T073611\backtest_res...,5.5,AAVEUSDT,True,-11.590,-0.05940,100.0,0.7649,True,54.850,0.2140,100.000,0.78480
3,\AKROUSDT\plots\2022-03-07T074200\backtest_res...,5.5,AKROUSDT,True,17.780,0.07660,100.0,0.9507,True,19.350,0.0738,100.000,0.96330
4,\API3USDT\plots\2022-03-14T161522\backtest_res...,5.5,API3USDT,True,7.298,0.21700,100.0,0.9586,True,7.734,0.5920,100.000,0.96320
5,\ARUSDT\plots\2022-03-15T224228\backtest_resul...,5.5,ARUSDT,True,10.490,0.05110,100.0,0.9353,True,27.360,0.1520,100.000,0.86110
6,\BAKEUSDT\plots\2022-03-14T161125\backtest_res...,5.5,BAKEUSDT,True,11.340,0.04540,100.0,0.9294,True,26.210,0.1060,100.000,0.93450
7,\CELRUSDT\plots\2022-03-15T224947\backtest_res...,5.5,CELRUSDT,True,22.550,0.08760,100.0,0.9427,True,0.000,0.5260,6.289,0.07394
8,\DGBUSDT\plots\2022-03-07T185158\backtest_resu...,5.5,DGBUSDT,True,11.750,0.05180,100.0,0.9583,True,22.520,0.0890,100.000,0.93090
9,\DGBUSDT\plots\2022-03-07T211718\backtest_resu...,5.5,DGBUSDT,True,11.750,0.05180,100.0,0.9583,True,22.520,0.0890,100.000,0.93090


Now filter the dataframe

In [4]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
                   "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
                   "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
                   "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
                   "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
                   "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
                   "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
                   "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
                   "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
                   "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
                   "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
                   "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
                   "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
                   "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
                   "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
                   "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
                   "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
                   "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
                   "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
                   "BTSUSDT", "DGBUSDT", "DEFIUSDT"]


allowed_symbols = ["AR","CELR","1000XEC","1000BTT","ROSE",]
allowed_symbols = [s + "USDT" for s in allowed_symbols]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
              & (big_data['long'] == True)
              & (big_data['short'] == True)
              & (big_data['adgt_%'] > 0.1)
              & (big_data['bnkrl_%'] > 95.0)
              & (big_data['bnkrs_%'] > 95.0)
              & (big_data['ebrs'] > .85)
              & (big_data['ebrl'] > .85)
              & (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
              ]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs,adgt_%
1,\1000XECUSDT\plots\2022-03-15T184506\backtest_...,5.5,1000XECUSDT,True,13.140,0.0652,100.0,0.9340,True,29.91,0.154,100.0,0.9526,0.2192
5,\ARUSDT\plots\2022-03-15T224228\backtest_resul...,5.5,ARUSDT,True,10.490,0.0511,100.0,0.9353,True,27.36,0.152,100.0,0.8611,0.2031
24,\ROSEUSDT\plots\2022-03-15T185118\backtest_res...,5.5,ROSEUSDT,True,5.621,0.0671,100.0,0.9516,True,12.92,0.179,100.0,0.9304,0.2461


In [5]:
df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',
                   inplace=True)
dx = df[['file','s','adgt_%','adgl_%','adgs_%','bnkrl_%', 'bnkrs_%','ebrl', 'ebrs']]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_102576/2043545472.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',


,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs
1,\1000XECUSDT\plots\2022-03-15T184506\backtest_...,1000XECUSDT,0.2192,0.0652,0.154,100.0,100.0,0.9340,0.9526
5,\ARUSDT\plots\2022-03-15T224228\backtest_resul...,ARUSDT,0.2031,0.0511,0.152,100.0,100.0,0.9353,0.8611
24,\ROSEUSDT\plots\2022-03-15T185118\backtest_res...,ROSEUSDT,0.2461,0.0671,0.179,100.0,100.0,0.9516,0.9304
